<div style="text-align: center; width: 100%; height: auto; padding: 100px;">
    <span style="font-size: 50px; font-weight: bold; display: block;">Data Processing Code</span>
    <br>
    <br>
<div style="text-align: left;font-size: 30px; font-weight: bold;line-height: 1.2; ">Data Category 1: GDP and Final Expenditure Components</div>
    <br>
    <br>
    <br>
    <div style="text-align: left; font-size: 15px;">Realized by Mehdi Jaddour</div>
    <div style="text-align: left; font-size: 15px;">
        <a href="https://github.com/JMehdi2/Major-Economic-Developments-in-Turkey-1998-Mid2024" target="_blank">Project GitHub Repository</a>
    </div>
</div>

In [513]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import warnings
from pandas.errors import PerformanceWarning

#Suppress PerformanceWarning
warnings.filterwarnings("ignore", category=PerformanceWarning)

curr_dir = os.getcwd()

<span style="font-size: 30px;font-weight: bold;">Table of Contents</span>

>##  [1. Annual Data](#year_gdp)
>>### [1.1. Data Preparation](#year_gdp_prep)
>>### [1.2. Identification of Additional Final Expenditure Aggregates](#year_gdp_agg)
>>### [1.3. Calculation of Growth Rates](#year_gdp_gr)
>>### [1.4. Calculation of Implicit Price Deflators](#year_gdp_def)
>>### [1.5. Volume and Price Decomposition into Trend and Cyclical Components](#year_gdp_decomp)
>>>#### [1.5.1. Volume Decomposition](#year_gdp_decomp_volume)  
>>>#### [1.5.2. Price Decomposition](#year_gdp_decomp_price)  
>>### [1.6. Calculation of Additive Contributions to Growth](#year_gdp_contrib)  
>>### [1.7. Shares in Main Aggregates](#year_gdp_share)
>>### [1.8. GDP per Capita](#year_gdp_percapita)
>>### [1.9. Change in Stocks in Real Terms](#year_gdp_stocks)

>##  [2. Quarterly Data](#quart_gdp)
>>### [2.1. Data Preparation](#quart_gdp_prep)
>>### [2.2. Identification of Additional Final Expenditure Aggregates](#quart_gdp_agg)
>>>#### [2.2.1. Temporal Consistency Check](#quart_gdp_agg_check) 
>>>> [2.2.1.1. Volume Indices](#quart_gdp_agg_check_vol)  
>>>> [2.2.1.2. Values at Current Prices](#quart_gdp_agg_check_nom)  
>>>#### [2.2.2. Calculation of Quarterly Additional Expenditure Aggregates](#quart_gdp_agg_quart)
>>### [2.3. Calculation of Growth Rates](#quart_gdp_gr) 
>>### [2.4. Calculation of Implicit Price Deflators](#quart_gdp_def)
>>### [2.5. Volume and Price Decomposition into Trend and Cyclical Components](#quart_gdp_decomp)
>>>#### [2.5.1. Volume Decomposition](#quart_gdp_decomp_volume)  
>>>#### [2.5.2. Price Decomposition](#quart_gdp_decomp_price)  
>>### [2.6. Calculation of Additive Contributions to QonQ and YoY Growth](#quart_gdp_contrib_qonq) 
>>### [2.7. Shares in Main Aggregates](#quart_gdp_share)
>>### [2.8. Change in Stocks in Real Terms](#quart_gdp_stocks)

<a class="anchor" id="year_gdp"></a>
# 1. Annual data

<a class="anchor" id="year_gdp_prep"></a>
## 1.1. Data Preparation 

In [514]:
#Load Data
##Volume index
year_gdp_volume = pd.read_excel('y_gdp_exp_cli.xls',index_col=0)
##Volume index, calendar adjusted
year_gdp_volume_caladj = pd.read_excel('y_gdp_exp_cli _caladj.xls',index_col=0)
##Value at current prices
year_gdp_nominal = pd.read_excel('y_gdp_exp_curr.xls',index_col=0)
##GDP per capita
year_percap_gdp = pd.read_excel('y_gdp_percapita_curr.xls',index_col=0)

In [515]:
#Initiate the Annual Parent Dataset
Datasets = [year_gdp_volume,year_gdp_nominal,year_gdp_volume_caladj]
keys = [('Accounts','Volume'),('Accounts','Nominal'),('Accounts','Volume Cal Adjusted')]
year_gdp_data = pd.concat(Datasets,axis=1,keys=keys)

<a class="anchor" id="year_gdp_agg"></a>
## 1.2. Identification of Additional Final Expenditure Aggregates 

In [516]:
#Define a function to aggregate volume indices and values at current prices
def y_aggregate_cli_nomi(comp1,comp2,agg_name):
    
    global year_gdp_data
    #Calculate value at current prices for the aggregate
    agg_nominal = year_gdp_data['Accounts','Nominal',comp1]+year_gdp_data['Accounts','Nominal',comp2]
    #Insert in parent dataset
    try:
        pos_nominal = year_gdp_data.columns.get_loc(('Accounts','Nominal',comp2))
        year_gdp_data.insert(pos_nominal+1,('Accounts','Nominal',agg_name),agg_nominal)
    except:
        year_gdp_data['Accounts','Nominal',agg_name] = agg_nominal
    #Aggregation of unadjusted and calendar adjusted volume indices
    for vol_adj in ['Volume','Volume Cal Adjusted']:
        #Calculate and aggregate volume indices from previous year
        agg_pyp = 0
        for comp in [comp1,comp2]:
            #Calculate the volume index from previous year for comp
            comp_pyp = year_gdp_data[('Accounts',vol_adj,comp)]/\
                year_gdp_data[('Accounts',vol_adj,comp)].shift()
            #Aggregate volume indices from previous year
            agg_pyp += comp_pyp*(year_gdp_data['Accounts','Nominal',comp]/agg_nominal).shift()
        #Chain Link the volume indices from previous year for the aggregate
        agg_cli = agg_pyp.cumprod()
        ##Assign 1 to year 1998
        agg_cli.loc[1998] = 1
        #Rereference to year 2009 and express on a scale of 100
        agg_cli = 100*agg_cli/agg_cli.loc[2009]
        #Insert in dataset
        try:
            pos_cli = year_gdp_data.columns.get_loc(('Accounts',vol_adj,comp2))
            year_gdp_data.insert(pos_cli+1,('Accounts',vol_adj,agg_name),agg_cli)
        except:
            year_gdp_data['Accounts',vol_adj,agg_name] = agg_cli
        
    print('Done')

In [517]:
#Identify aggregates
#Final Private Consumption Expenditure, Final Domestic Consumption Expenditure, and Final Domestic Demand Expenditure 
#(see A-III-3 in the Methods Section of the report)

L = [('Final Consumption Expenditure of Households','Final Consumption Expenditure Of Npish','Final Private Consumption Expenditure'),
('Final Private Consumption Expenditure','Government Final Consumption Expenditure','Final Domestic Consumption Expenditure'),
('Final Domestic Consumption Expenditure','Gross Fixed Capital Formation','Final Domestic Demand Expenditure')]

for req in L:
    y_aggregate_cli_nomi(req[0],req[1],req[2])

Done
Done
Done


 <a class="anchor" id="year_gdp_gr"></a>
## 1.3. Calculation of Growth Rates

In [518]:
#Calculate Growth Rates
year_gdp_growth = year_gdp_data['Accounts'].diff()/abs(year_gdp_data['Accounts'].shift())
#Concatenate
year_gdp_growth = pd.concat([year_gdp_growth],axis=1,keys=['Growth Rate'])
year_gdp_data = pd.concat([year_gdp_data,year_gdp_growth],axis=1)

 <a class="anchor" id="year_gdp_def"></a>
## 1.4. Calculation of Implicit Price Deflators

In [519]:
for col in year_gdp_data['Accounts','Volume'].columns: 
    #Calculate the Implicit Price Deflator
    year_gdp_data['Price Deflator',col,'Index'] = 100*year_gdp_data['Accounts','Nominal',col]/\
    (year_gdp_data['Accounts','Volume',col]/100*year_gdp_data['Accounts','Nominal',col].loc[2009])
    #Calculate the YonY inflation rate
    year_gdp_data['Price Deflator',col,'YonY %'] = year_gdp_data['Price Deflator',col,'Index'].diff()/year_gdp_data['Price Deflator',col,'Index'].shift()
    
for col in year_gdp_data['Accounts','Volume'].columns: 
    #Calculate the Implicit Price Deflator
    year_gdp_data['Price Deflator Cal Adjusted',col,'Index'] = 100*year_gdp_data['Accounts','Nominal',col]/\
    (year_gdp_data['Accounts','Volume Cal Adjusted',col]/100*year_gdp_data['Accounts','Nominal',col].loc[2009])
    #Calculate the YonY inflation rate
    year_gdp_data['Price Deflator Cal Adjusted',col,'YonY %'] = year_gdp_data['Price Deflator Cal Adjusted',col,'Index'].diff()/year_gdp_data['Price Deflator Cal Adjusted',col,'Index'].shift()
    

<a class="anchor" id="year_gdp_decomp"></a>
## 1.5. Volume and Price Decomposition into Trend and Cyclical Components

<a class="anchor" id="year_gdp_decomp_volume"></a>
### 1.5.1. Volume Decomposition

In [520]:
for col in year_gdp_data['Accounts','Volume Cal Adjusted'].columns:
    #Calculate log values
    series_log = np.log(year_gdp_data['Accounts','Volume Cal Adjusted',col])
    year_gdp_data['Volume Decomp',col,'Log Value'] = series_log
    #Decompose using the HP Filter
    series_cyc ,series_trend  = sm.tsa.filters.hpfilter(series_log.loc[:2023],lamb=6.25)
    #Reverse Log transformation to get the trend for the original series
    year_gdp_data['Volume Decomp',col,'Trend'] = np.exp(series_trend)
    #Calculate the cyclical factor for the original series
    year_gdp_data['Volume Decomp',col,'Cyclical'] = year_gdp_data['Accounts','Volume Cal Adjusted',col]/np.exp(series_trend)
    #Calculate the percentage deviation from the trend
    year_gdp_data['Volume Decomp',col,'Percentage Deviation'] = year_gdp_data['Volume Decomp',col,'Cyclical']-1
    #Trend growth rate
    year_gdp_data['Volume Decomp',col,'Trend Growth Rate'] = np.exp(series_trend).diff()/np.exp(series_trend).shift()

<a class="anchor" id="year_gdp_decomp_price"></a>
### 1.5.2 Price Decomposition

In [521]:
for col in year_gdp_data['Price Deflator Cal Adjusted'].columns.get_level_values(0).unique():
    #Calculate log values
    series_log = np.log(year_gdp_data['Price Deflator Cal Adjusted',col,'Index'])
    year_gdp_data['Price Decomp',col,'Log Value'] = series_log
    #Decompose using the HP Filter
    series_cyc ,series_trend  = sm.tsa.filters.hpfilter(series_log.loc[:2023],lamb=6.25)
    #Reverse Log transformation to get the trend for the original series
    year_gdp_data['Price Decomp',col,'Trend'] = np.exp(series_trend)
    #Calculate the cyclical factor for the original series
    year_gdp_data['Price Decomp',col,'Cyclical'] = year_gdp_data['Price Deflator Cal Adjusted',col,'Index']/np.exp(series_trend)
    #Calculate the percentage deviation from the trend
    year_gdp_data['Price Decomp',col,'Percentage Deviation'] = year_gdp_data['Price Decomp',col,'Cyclical']-1
    #Trend growth rate
    year_gdp_data['Price Decomp',col,'Trend Growth Rate'] = np.exp(series_trend).diff()/np.exp(series_trend).shift()

<a class="anchor" id="year_gdp_contrib"></a>
## 1.6. The Calculation of Additive Contributions to Growth 

In [522]:
#Prepare the required series (Price Indices and Volume measures in monetary terms)
price_index = year_gdp_data.loc[:,('Price Deflator',slice(None),'Index')].copy()
price_index.columns = price_index.columns.droplevel([0,2])
year_gdp_cli_money = 0.01*year_gdp_data['Accounts','Volume'].multiply(year_gdp_data.loc[2009,('Accounts','Nominal')],axis=1)

In [523]:
#Define a function that computes contributions to nominal and volume growth
def contrib(Comp,agg_name):
    
    global year_gdp_data,year_gdp_cli_money,price_index
    #Calculate contributions to volume growth
    df_v = year_gdp_cli_money[list(Comp)].diff()*price_index[list(Comp)].shift().div((year_gdp_cli_money[Comp[0]]*price_index[Comp[0]]).shift(),axis=0)
   
    #Calculate contributions to nominal growth
    if agg_name == 'GDP':
        df_n = year_gdp_data['Accounts','Nominal'].diff().div(year_gdp_data['Accounts','Nominal','GDP'].shift(),axis=0)
    else:
        df_n = year_gdp_data['Accounts','Nominal'][list(Comp)].diff().div(year_gdp_data['Accounts','Nominal',Comp[0]].shift(),axis=0)
    
    ## For contributions to GDP growth
    if agg_name == 'GDP':
        ###Calculate the contribution of Change In stocks to volume growth as a residual
        df_v['Change In stocks'] = df_v['GDP'] - (df_v['Final Domestic Demand Expenditure'] + \
                        df_v['Exports Of Goods And Services'] - df_v['Imports Of Goods And Services'])
        ###Adjust the sign for the contribution of Imports to volume and nominal growths
        df_v['Imports Of Goods And Services'] = - df_v['Imports Of Goods And Services']
        df_n['Imports Of Goods And Services'] = - df_n['Imports Of Goods And Services']
        
    #Concatenate contribution datasets
    key_0 = f'Contribution to {agg_name} Growth'
    df = pd.concat([df_v,df_n],keys=[(key_0,'Volume'),(key_0,'Nominal')],axis=1)
    #Add to parent dataset
    year_gdp_data = pd.concat([year_gdp_data,df],axis=1)

In [524]:
#Compute contributions to the growth of:
#GDP, Final Domestic Consumption Expenditure, and Final Domestic Demand Expenditure 
#(see A-III-6 in the Methods Section of the report)
S1 = tuple(year_gdp_data['Accounts','Volume'].columns)
S2 = ('Final Domestic Consumption Expenditure','Final Consumption Expenditure of Households',
        'Final Consumption Expenditure Of Npish','Final Private Consumption Expenditure',
    'Government Final Consumption Expenditure')
S3 = ('Final Domestic Demand Expenditure','Final Consumption Expenditure of Households',
        'Final Consumption Expenditure Of Npish','Final Private Consumption Expenditure',
    'Government Final Consumption Expenditure','Final Domestic Consumption Expenditure','Gross Fixed Capital Formation')

L = [(S1,'GDP'),(S2,'Domestic Consumption'),(S3,'Domestic Demand')]

for req in L:
    contrib(req[0],req[1])

<a class="anchor" id="year_gdp_share"></a>
## 1.7. Shares in Main Aggregates

In [525]:
#Prepare the required data (price indices and volume measures in monetary terms)
price_index = year_gdp_data.loc[:,('Price Deflator',slice(None),'Index')].copy()
price_index.columns = price_index.columns.droplevel([0,2])
year_gdp_cli_money = 0.01*year_gdp_data['Accounts','Volume'].multiply(year_gdp_data.loc[2009,('Accounts','Nominal')],axis=1)

In [526]:
def share(Comp,name):
    
    global year_gdp_data,price_index,year_gdp_cli_money
    #Calculate nominal shares
    if name == 'GDP':
        df_n = year_gdp_data['Accounts','Nominal'].div(year_gdp_data['Accounts','Nominal','GDP'],axis=0)
    else:
        df_n = year_gdp_data['Accounts','Nominal'][list(Comp)].div(year_gdp_data['Accounts','Nominal',Comp[0]],axis=0)
        
    #Calculate the volume and price components of the nominal shares
    df_v = year_gdp_cli_money[list(Comp)].div(year_gdp_cli_money[Comp[0]],axis=0)
    df_p = price_index[list(Comp)].div(price_index[Comp[0]],axis=0)
    
    ## For contributions to GDP growth
    if name == 'GDP':
        ###Adjust the sign for the share and volume component of Imports
        df_v['Imports Of Goods And Services'] = - df_v['Imports Of Goods And Services']
        df_n['Imports Of Goods And Services'] = - df_n['Imports Of Goods And Services']
        
    #Concatenate shares and components
    key_0 = f'Share in {name}'
    df = pd.concat([df_n,df_v,df_p],keys=[(key_0,'Nominal'),(key_0,'Relative Volume'),(key_0,'Relative Price')],axis=1)
    #Add to parent dataset
    year_gdp_data = pd.concat([year_gdp_data,df],axis=1)

In [527]:
#Compute shares in:
#GDP, Final Domestic Consumption Expenditure, and Final Domestic Demand Expenditure 
#(see A-III-7 in the Methods Section of the report)
S1 = tuple(year_gdp_data['Accounts','Volume'].columns)
S2 = ('Final Domestic Consumption Expenditure','Final Consumption Expenditure of Households',
        'Final Consumption Expenditure Of Npish','Final Private Consumption Expenditure',
    'Government Final Consumption Expenditure')
S3 = ('Final Domestic Demand Expenditure','Final Consumption Expenditure of Households',
        'Final Consumption Expenditure Of Npish','Final Private Consumption Expenditure',
    'Government Final Consumption Expenditure','Final Domestic Consumption Expenditure','Gross Fixed Capital Formation')

L = [(S1,'GDP'),(S2,'Domestic Consumption'),(S3,'Domestic Demand')]

for req in L:
    share(req[0],req[1])

<a class="anchor" id="year_gdp_percapita"></a>
## 1.8. GDP Per Capita

In [528]:
#Calculate growth rates for nominal GDP per capita
year_percap_gdp['TRY value Growth'] = year_percap_gdp['TRY value'].diff()/ year_percap_gdp['TRY value'].shift()
year_percap_gdp['USD value Growth'] = year_percap_gdp['USD value'].diff()/ year_percap_gdp['USD value'].shift()
#Add column index level
year_percap_gdp = pd.concat([year_percap_gdp],keys=['Nominal'],axis=1)

In [529]:
#Calculate GDP per capita's volume Index and its Growth Rate
##Unadjusted
year_percap_gdp['Volume','Volume'] = 100*(year_percap_gdp['Nominal','TRY value']/year_percap_gdp.loc[2009,('Nominal','TRY value')])/(year_gdp_data['Price Deflator','GDP','Index']/100)
year_percap_gdp['Volume','Volume Growth'] = year_percap_gdp['Volume','Volume'].diff()/year_percap_gdp['Volume','Volume'].shift()
##Calendar adjusted
year_percap_gdp['Volume','Volume Cal Adjusted'] = 100*(year_percap_gdp['Nominal','TRY value']/year_percap_gdp.loc[2009,('Nominal','TRY value')])/(year_gdp_data['Price Deflator Cal Adjusted','GDP','Index']/100)
year_percap_gdp['Volume','Volume Cal Adjusted Growth'] = year_percap_gdp['Volume','Volume Cal Adjusted'].diff()/year_percap_gdp['Volume','Volume Cal Adjusted'].shift()

In [530]:
#Concatenate
year_percap_gdp = pd.concat([year_percap_gdp],keys=['GDP per Capita'],axis=1)
year_gdp_data = pd.concat([year_gdp_data,year_percap_gdp],axis=1)

<a class="anchor" id="year_gdp_stocks"></a>
## 1.9. Change in Stocks in Real Terms

In [531]:
#For unadjusted and adjusted series (nominal series do not contain calendar effects)
for nat in [('Volume','Price Deflator'),('Volume Cal Adjusted','Price Deflator Cal Adjusted')]:
    #Calculate change in stocks in real terms
    series_r = 100*(year_gdp_data['Accounts','Nominal','Change In stocks']/year_gdp_data.loc[2009,('Accounts','Nominal','Change In stocks')])\
/(0.01*year_gdp_data[nat[1],'GDP','Index'])
    #Calculate growth rate
    series_gr = series_r.diff()/abs(series_r.shift())
    #Insert in parent dataset
    Series = [series_r,series_gr]
    j = 0
    for data in ['Accounts','Growth Rate']:
        pos = year_gdp_data.columns.get_loc((data,nat[0],'Final Domestic Demand Expenditure'))+1
        year_gdp_data.insert(pos,(data,nat[0],'Change in Stocks (Real)'),Series[j])
        j += 1

### Save Data

file_name = curr_dir + '\\Results\\y_gdp_data.xlsx'
year_gdp_data.to_excel(file_name)

# 2. Quarterly data <a class="anchor" id="quart_gdp"></a>

## 2.1. Data Preparation <a class="anchor" id="quart_gdp_prep"></a>

In [532]:
#Load Data
##Adjusted Volume Index
quart_gdp_volume_alladj = pd.read_excel('q_gdp_cli_adj.xls',index_col=[0,1])
##Unadjusted Volume Index
quart_gdp_volume_noadj = pd.read_excel('q_gdp_cli_noadj.xls',index_col=[0,1])
##Seasonally Adjusted Volume Index
quart_gdp_volume_seaadj = pd.read_excel('q_gdp_cli_seaadj.xls',index_col=[0,1])
##Seasonally Adjusted Value at Current Prices
quart_gdp_nominal_seaadj = pd.read_excel('q_gdp_currpr_seaadj.xls',index_col=[0,1])
##Unadjusted Value at Current Prices
quart_gdp_nominal_noadj = pd.read_excel('q_gdp_currpr_noadj.xls',index_col=[0,1])

**Concatenate all in one dataframe**

In [533]:
#Initiate the quarterly parent dataset
Datasets = [quart_gdp_volume_alladj,quart_gdp_volume_seaadj,quart_gdp_volume_noadj,quart_gdp_nominal_seaadj,
            quart_gdp_nominal_noadj]
keys = [('Accounts','Volume','Adjusted'),('Accounts','Volume','Seas Adjusted'),('Accounts','Volume','Not Adjusted'),
            ('Accounts','Nominal','Seas Adjusted'),('Accounts','Nominal','Not Adjusted')]

quart_gdp_data = pd.concat(Datasets,axis=1,keys=keys) 

<a class="anchor" id="quart_gdp_agg"></a>
## 2.2. Identification of Additional Final Expenditure Aggregates 

<a class="anchor" id="quart_gdp_agg_check"></a>
### 2.2.1. Temporal Consistency Check 

<a class="anchor" id="quart_gdp_agg_check_vol"></a>\
**2.2.1.1. Volume Indices** 

In [534]:
#Create a plot for the differences between the temporal aggregate series and the annual series, by adjustments (use the dropdown menu)
#Create a statistical summary of the results (results_1)

def create_figure_summary(choice):
    
    if choice == 'Unadjusted': vol_adj = 'Volume'
    else: vol_adj = 'Volume Cal Adjusted'
    
    ##Temporally aggregate quarterly series
    year_agg = quart_gdp_data.groupby('Year').mean()
    
    #Dataframe to store results summary
    index = [(i,j) for i in ['Not Adjusted','Adjusted','Seas Adjusted'] for j in quart_gdp_data['Accounts','Volume','Not Adjusted'].columns]
    criteria = ['Difference in Indices','Difference in monetary terms','Percentage Deviation']
    statistics = ['Average','Max']
    columns = [(i,j) for i in criteria for j in statistics]
    multi_ind =  pd.MultiIndex.from_tuples(index)
    multi_col =  pd.MultiIndex.from_tuples(columns)
    ##Dataframe
    results_1 = pd.DataFrame(index=multi_ind,columns=multi_col)
    
    #Plot
    fig = make_subplots(rows=3,cols=3,subplot_titles=['Not Adjusted','Seas Adjusted','Seas and Cal Adjusted'])
    colors = ['blue', 'orange', 'green', 'red', 'purple','magenta' ,'brown', 'gray','purple','yellow','pink', 'olive', 'cyan','silver',
         'indigo','orange','green','red']
    i=0
    for col in quart_gdp_data['Accounts','Volume','Not Adjusted'].columns:
        j=0
        for adj in ['Not Adjusted','Seas Adjusted','Adjusted']:
            #Calculate the absolute difference between the indices
            series_diff = abs(year_agg['Accounts','Volume',adj,col]-year_gdp_data['Accounts',vol_adj,col])
            #Calculate the absolute difference in monetary terms
            series_diff_money = 0.01*series_diff * year_gdp_data.loc[2009,('Accounts','Nominal',col)]
            #Calculate the absolute percentage deviation of the temporally aggregated indices from the annual indices
            series_dev = 100*series_diff/year_gdp_data['Accounts',vol_adj,col]
            #Calculate maximums and means
            stats = [(series_diff.mean(),series_diff.max()),(series_diff_money.mean(),series_diff_money.max()),
                     (series_dev.mean(),series_dev.max())]
            #Create Scatter objects
            trace_diff = go.Scatter(x=year_gdp_data.index,y=series_diff,name=col,
                                    line=dict(color=colors[i]),legendgroup=f'group{i}',showlegend=(j==0))
            trace_diff_money = go.Scatter(x=year_gdp_data.index,y=series_diff_money,name=col,
                                    line=dict(color=colors[i]),legendgroup=f'group{i}',showlegend=False)
            trace_dev = go.Scatter(x=year_gdp_data.index,y=series_dev,name=col,
                                   line=dict(color=colors[i]),legendgroup=f'group{i}',showlegend=False)
            Trace = [trace_diff,trace_diff_money,trace_dev]
            #Add statistics to dataframe and Scatters to graph
            for p in range(len(criteria)):
                results_1.loc[(adj,col),(criteria[p],'Average')] = stats[p][0]
                results_1.loc[(adj,col),(criteria[p],'Max')] = stats[p][1]
                fig.add_trace(Trace[p],row=1+p,col=1+j)

            j +=1
        i+=1
    #Update the layout of the graph
    for p in range(len(criteria)): fig.update_yaxes(title=f'Absolute {criteria[p]}'+' %'*(p==2),row=1+p,col=1)
    fig.update_layout(width=1250,height=900,title=f'The Difference between Temporally Aggregated Indices and {choice} Annual Indices',title_x=0.4)
    #Adjust datatype in the results summary dataframe
    results_1=results_1.astype('float64')
    
    return fig,results_1

#Dropdown widget
dropdown = widgets.Dropdown(
    options=['Unadjusted', 'Calendar Adjusted'],
    value='Unadjusted',
    description='Adjustment applied to the annual series:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='initial') )
#Output widget
graph_output = widgets.Output()
df_output = widgets.Output()
#Function to update the graph
def update(change):
    with graph_output:
        graph_output.clear_output(wait=True)
        fig,results_1 = create_figure_summary(change['new'])
        fig.show()
    with df_output:
        df_output.clear_output(wait=True)
        display(results_1)
# Link the dropdown widget to the update function
dropdown.observe(update, names='value')
# Display the widgets and initial outputs
display(dropdown, graph_output, df_output)
# Initialize with the first function
update({'new': dropdown.value})

Dropdown(description='Adjustment applied to the annual series:', layout=Layout(width='initial'), options=('Una…

Output()

Output()

<a class="anchor" id="quart_gdp_agg_check_nom"></a>\
**2.2.1.2. Values at Current Prices** 

In [535]:
#Settings for the results summary and graph

##Dataframe to store results summary
index = [(i,j) for i in ['Not Adjusted','Seas Adjusted'] for j in quart_gdp_data['Accounts','Nominal','Not Adjusted'].columns]
criteria = ['Difference in Values','Percentage Deviation']
statistics = ['Average','Max']
columns = [(i,j) for i in criteria for j in statistics]
multi_ind =  pd.MultiIndex.from_tuples(index)
multi_col =  pd.MultiIndex.from_tuples(columns)
results_2 = pd.DataFrame(index=multi_ind,columns=multi_col)
##Temporally aggregate quarterly series
year_agg = quart_gdp_data.groupby('Year').sum()

#Create a plot for the differences between the temporal aggregate series and the annual series by adjustments
#Create a statistical summary of the results (results_2)

colors = ['blue', 'orange', 'green', 'red', 'purple','magenta' ,'brown', 'gray','purple','yellow','pink', 'olive', 'cyan','silver',
         'indigo','orange','green','red']
fig = make_subplots(rows=2,cols=2,subplot_titles=['Not Adjusted','Seas Adjusted'])
i=0
for col in quart_gdp_data['Accounts','Nominal','Not Adjusted'].columns:
    j=0
    for adj in ['Not Adjusted','Seas Adjusted']:
        #Calculate the absolute difference between the values at current prices
        series_diff = abs(year_agg['Accounts','Nominal',adj,col]-year_gdp_data['Accounts','Nominal',col])
        #Calculate the absolute percentage deviation of the temporally aggregated values from the annual ones
        series_dev = 100*series_diff/abs(year_gdp_data['Accounts','Nominal',col])
        #Calculate maximums and means
        stats = [(series_diff.mean(),series_diff.max()),(series_dev.mean(),series_dev.max())]
        #Create Scatter objects
        trace_diff = go.Scatter(x=year_gdp_data.index,y=series_diff,name=col,
                                line=dict(color=colors[i]),legendgroup=f'group{i}',showlegend=(j==0))
        trace_dev = go.Scatter(x=year_gdp_data.index,y=series_dev,name=col,
                               line=dict(color=colors[i]),legendgroup=f'group{i}',showlegend=False)
        Trace = [trace_diff,trace_dev]
        #Add statistics to dataframe and Scatters to graph
        for p in range(len(criteria)):
            results_2.loc[(adj,col),(criteria[p],'Average')] = stats[p][0]
            results_2.loc[(adj,col),(criteria[p],'Max')] = stats[p][1]
            fig.add_trace(Trace[p],row=1+p,col=1+j)
        j +=1
    i+=1
#Update the layout
fig.update_yaxes(title='Absolute difference in thousand TRY',row=1,col=1)
fig.update_yaxes(title='Absolute Percentage deviation (%)',row=2,col=1)
fig.update_layout(width=1250,height=600,title='The Difference between Temporally Aggregated Nominal Values and Unadjusted Annual Nominal Values',title_x=0.4)
#Adjust datatype
results_2=results_2.astype('float64')

fig.show() 
display(results_2)

Difference in Values  \
                                                                       Average   
Not Adjusted  GDP                                                 2.937477e-05   
              Final Consumption Expenditure of Households         1.194672e-06   
              Final Consumption Expenditure Of Npish              5.038321e-09   
              Government Final Consumption Expenditure            8.091760e-08   
              Gross Fixed Capital Formation                       9.353344e-06   
              Change In stocks                                    2.009055e-05   
              Exports Of Goods And Services                       5.201078e-07   
              Imports Of Goods And Services                       8.518736e-07   
Seas Adjusted GDP                                                 2.684502e-05   
              Final Consumption Expenditure of Households         7.868615e-05   
              Final Consumption Expenditure Of Npish              7.960272e-05   
              Government Final Consumption Expenditure            1.134297e-04   
              Gross Fixed Capital Formation                       4.431419e-05   
              Change In stocks                                    3.555593e-04   
              Exports Of Goods And Services                       5.280119e-05   
              Imports Of Goods And Services                       6.794930e-05   

                                                                         \
                                                                    Max   
Not Adjusted  GDP                                          4.014969e-04   
              Final Consumption Expenditure of Households  3.695488e-06   
              Final Consumption Expenditure Of Npish       3.539026e-08   
              Government Final Consumption Expenditure     9.536743e-07   
              Gross Fixed Capital Formation                1.945496e-04   
              Change In stocks                             3.994927e-04   
              Exports Of Goods And Services                2.861023e-06   
              Imports Of Goods And Services                5.722046e-06   
Seas Adjusted GDP                                          3.900528e-04   
              Final Consumption Expenditure of Households  2.522469e-04   
              Final Consumption Expenditure Of Npish       6.761402e-04   
              Government Final Consumption Expenditure     2.775192e-04   
              Gross Fixed Capital Formation                1.688004e-04   
              Change In stocks                             1.438230e-03   
              Exports Of Goods And Services                1.211166e-04   
              Imports Of Goods And Services                2.188683e-04   

                                                          Percentage Deviation  \
                                                                       Average   
Not Adjusted  GDP                                                 1.213638e-12   
              Final Consumption Expenditure of Households         3.431078e-13   
              Final Consumption Expenditure Of Npish              9.459920e-14   
              Government Final Consumption Expenditure            1.125387e-14   
              Gross Fixed Capital Formation                       1.933922e-12   
              Change In stocks                                    1.261191e-10   
              Exports Of Goods And Services                       2.760385e-13   
              Imports Of Goods And Services                       2.656406e-13   
Seas Adjusted GDP                                                 6.051201e-13   
              Final Consumption Expenditure of Households         1.191585e-11   
              Final Consumption Expenditure Of Npish              2.663854e-09   
              Government Final Consumption Expenditure            1.033908e-10   
              Gross Fixed Capital Formation                       2.025907e-11   


### 2.2.2. Calculation of Quarterly Additional Expenditure Aggregates <a class="anchor" id="quart_gdp_agg_quart"></a>

In [536]:
#See A-III-3 in the Methods section of the report

#Define a function to derive annual links from quarterly volume indices from previous year
def annual_links(quart_pyp,Insert=(None,None)):
    
    global year_gdp_data
    #Calculate annual links based on quarterly volume indices from previous year
    year_pyp = quart_pyp.groupby('Year').mean()
    year_pyp.loc[2024] = np.nan
    #Chain-link annual links to year 2009
    year_cli = year_pyp.cumprod()
    year_cli.loc[1998] = 1
    year_cli = 100*year_cli/year_cli.loc[2009]
    #Add the chain-linked annual links to the annual parent dataset
    if Insert[0] and Insert[1]:
        year_gdp_data['Accounts',Insert[0],Insert[1]] = year_cli
        
    return year_cli

#Define a function to aggregate quarterly volume indices and values at current prices
def q_aggregate_cli_nomi(comp1,comp2,agg_name,Inconsistent=[None]):
    
    global year_gdp_data,quart_gdp_data
    #Aggregation of unadjusted and seasonally adjusted values at current prices
    for adj in ['Not Adjusted','Seas Adjusted']:
        agg_nominal = quart_gdp_data['Accounts','Nominal',adj,comp1]+quart_gdp_data['Accounts','Nominal',adj,comp2]
        #Insert in parent dataset
        try:
            pos_nominal = quart_gdp_data.columns.get_loc(('Accounts','Nominal',adj,comp2))
            quart_gdp_data.insert(pos_nominal+1,('Accounts','Nominal',adj,agg_name),agg_nominal)
        except:
            quart_gdp_data['Accounts','Nominal',adj,agg_name] = agg_nominal
    
    #Aggregation of unadjusted and seasonally and calendar adjusted volume indices
    for adj in ['Not Adjusted','Adjusted']:
        #Calculate and aggregate volume indices from previous year
        agg_pyp = 0
        for comp in [comp1,comp2]:
            if (comp,adj) in Inconsistent:
                try:
                    #Check if the chain-linked annual links are already calculated
                    comp_link = year_gdp_data['Accounts','Volume'*(adj=='Not Adjusted')+'Volume Cal Adjusted'*(adj=='Adjusted'),f'{comp} 2']
                except:
                    #Derive the chain-linked annual links from the quarterly volume indices 
                    comp_link = quart_gdp_data['Accounts','Volume',adj,comp].groupby('Year').mean()
                    comp_link.loc[2024] = np.nan
                    #Add the chain-linked annual links to the annual parent dataset
                    year_gdp_data['Accounts','Volume'*(adj=='Not Adjusted')+'Volume Cal Adjusted'*(adj=='Adjusted'),f'{comp} 2'] =\
                    comp_link
            else: #If the quarterly and annual volume index series are temporally consistent, then the annual links are equal to the annual volume indices
                comp_link = year_gdp_data['Accounts','Volume'*(adj=='Not Adjusted')+'Volume Cal Adjusted'*(adj=='Adjusted'),comp]
            #Calculate the volume index from previous year for comp
            comp_pyp = quart_gdp_data['Accounts','Volume',adj,comp]\
            /comp_link.shift()
            #Aggregating (based on the fact that values at current prices are temporally consistent)
            agg_pyp += comp_pyp*(year_gdp_data['Accounts','Nominal',comp]/year_gdp_data['Accounts','Nominal',agg_name]).shift()
        
        #Chain_link the aggregate indices from previous year
        if (agg_name,adj) in Inconsistent:
            #Calculate the chain-linked annual links
            agg_link = annual_links(agg_pyp,Insert=('Volume'*(adj=='Not Adjusted')+'Volume Cal Adjusted'*(adj=='Adjusted'),f'{agg_name} 2'))
        else: #If the quarterly and annual volume index series are temporally consistent, then the annual links are equal to the annual volume indices
            agg_link = year_gdp_data['Accounts','Volume'*(adj=='Not Adjusted')+'Volume Cal Adjusted'*(adj=='Adjusted'),agg_name]
        agg_cli = agg_pyp * agg_link.shift()
        #Insert in parent dataset
        try:
            pos_cli = quart_gdp_data.columns.get_loc(('Accounts','Volume',adj,comp2))
            quart_gdp_data.insert(pos_cli+1,('Accounts','Volume',adj,agg_name),agg_cli)
        except:
            quart_gdp_data['Accounts','Volume',adj,agg_name] = agg_cli
    
    print('Done')

In [537]:
#Identify aggregates
#Final Private Consumption Expenditure, Final Domestic Consumption Expenditure, and Final Domestic Demand Expenditure 
#(see A-III-3 in the Methods Section of the report)

L = [('Final Consumption Expenditure of Households','Final Consumption Expenditure Of Npish','Final Private Consumption Expenditure'),
('Final Private Consumption Expenditure','Government Final Consumption Expenditure','Final Domestic Consumption Expenditure'),
('Final Domestic Consumption Expenditure','Gross Fixed Capital Formation','Final Domestic Demand Expenditure')]
##The list of temporally inconsistent volume index series (See Methods section, in A-III-3.2.2.b, Step 0 and 3)
Inconsistent = [('Government Final Consumption Expenditure','Adjusted'),('Final Domestic Consumption Expenditure','Adjusted'),
             ('Final Domestic Demand Expenditure','Adjusted')  ]
for req in L:
    q_aggregate_cli_nomi(req[0],req[1],req[2],Inconsistent)

Done
Done
Done


<a class="anchor" id="quart_gdp_gr"></a>
## 2.3. Calculation of Growth Rates 

In [538]:
#QonQ and YoY growth rates
for freq in ['QonQ','YoY']:
    f = 1*(freq=='QonQ') + 4*(freq=='YoY')
    df = quart_gdp_data['Accounts'].diff(f)/abs(quart_gdp_data['Accounts'].shift(f))
    df = pd.concat([df],keys=[f'Growth Rate {freq}'],axis=1)
    quart_gdp_data = pd.concat([quart_gdp_data,df],axis=1)

<a class="anchor" id="quart_gdp_def"></a>
## 2.4. Calculation of Implicit Price Deflators

In [539]:
Adj = [('Adjusted','Seas Adjusted'),('Seas Adjusted','Seas Adjusted'),('Not Adjusted','Not Adjusted')]
for adj in Adj:
    for col in quart_gdp_data['Accounts','Volume',adj[0]].columns:
        #Calculate the implicit price deflator
        quart_gdp_data['Price Deflator',adj[0],col,'Index'] = 100*quart_gdp_data['Accounts','Nominal',adj[1],col]/\
        (quart_gdp_data['Accounts','Volume',adj[0],col]/100*year_gdp_data['Accounts','Nominal',col].loc[2009]/4)
        #Calculate the QonQ inflation rate
        quart_gdp_data['Price Deflator',adj[0],col,'QonQ %'] = quart_gdp_data['Price Deflator',adj[0],col,'Index'].diff()/quart_gdp_data['Price Deflator',adj[0],col,'Index'].shift()
        #Calculate the YoY inflation rate
        quart_gdp_data['Price Deflator',adj[0],col,'YoY %'] = quart_gdp_data['Price Deflator',adj[0],col,'Index'].diff(4)/quart_gdp_data['Price Deflator',adj[0],col,'Index'].shift(4)

<a class="anchor" id="quart_gdp_decomp"></a>
## 2.5.  Volume and Price Decomposition into Trend and Cyclical Components 

<a class="anchor" id="quart_gdp_decomp_volume"></a>
### 2.5.1. Volume Decomposition 

In [540]:
Exception = ['Final Private Consumption Expenditure', 'Final Domestic Consumption Expenditure','Final Domestic Demand Expenditure']

for col in quart_gdp_data['Accounts','Volume','Adjusted'].columns:
    
    #A conditional statement to manage missing 1998 data in the identified new aggregates
    if col in Exception: y=1999
    else: y=1998
    #Calculate Log values
    series_log = np.log(quart_gdp_data['Accounts','Volume','Adjusted',col])
    quart_gdp_data['Volume Decomp','Adjusted',col,'Log Value'] = series_log        
    #Use HP Filter
    series_cyc,series_trend = sm.tsa.filters.hpfilter(series_log.loc[y:],lamb=1600) 
    #Reverse Log transformation to get the trend for the original series
    quart_gdp_data['Volume Decomp','Adjusted',col,'Trend'] = np.exp(series_trend)
    #Calculate the cyclical factor for the original series
    quart_gdp_data['Volume Decomp','Adjusted',col,'Cyclical'] = quart_gdp_data['Accounts','Volume','Adjusted',col]/np.exp(series_trend)
    #Calculate the percentage deviation from trend
    quart_gdp_data['Volume Decomp','Adjusted',col,'Percentage Deviation'] = quart_gdp_data['Volume Decomp','Adjusted',col,'Cyclical']-1
    #Trend QonQ growth rate
    quart_gdp_data['Volume Decomp','Adjusted',col,'Trend QonQ Growth Rate'] = np.exp(series_trend).diff()/np.exp(series_trend).shift()
    #Trend YoY growth rate
    quart_gdp_data['Volume Decomp','Adjusted',col,'Trend YoY Growth Rate'] = np.exp(series_trend).diff(4)/np.exp(series_trend).shift(4)

 <a class="anchor" id="quart_gdp_decomp_price"></a>
### 2.5.2. Price Decomposition

In [541]:
Exception = ['Final Private Consumption Expenditure', 'Final Domestic Consumption Expenditure','Final Domestic Demand Expenditure']

for col in quart_gdp_data['Price Deflator','Adjusted'].columns.get_level_values(0).unique():

    #A conditional statement to manage missing 1998 data in the identified new aggregates
    if col in Exception: y=1999
    else: y=1998
    #Calculate Log values
    series_log = np.log(quart_gdp_data['Price Deflator','Adjusted',col,'Index'])
    quart_gdp_data['Price Decomp','Adjusted',col,'Log Value'] = series_log
    #Use HP Filter
    series_cyc,series_trend = sm.tsa.filters.hpfilter(series_log.loc[y:],lamb=1600) 
    #Reverse Log transformation to get the trend for the original series
    quart_gdp_data['Price Decomp','Adjusted',col,'Trend'] = np.exp(series_trend)
    #Calculate the cyclical factor for the original series
    quart_gdp_data['Price Decomp','Adjusted',col,'Cyclical'] = quart_gdp_data['Price Deflator','Adjusted',col,'Index']/np.exp(series_trend)
    #Calculate the percentage deviation from trend
    quart_gdp_data['Price Decomp','Adjusted',col,'Percentage Deviation'] = quart_gdp_data['Price Decomp','Adjusted',col,'Cyclical']-1
    #Trend QonQ growth rate
    quart_gdp_data['Price Decomp','Adjusted',col,'Trend QonQ Growth Rate'] = np.exp(series_trend).diff()/np.exp(series_trend).shift()
    #Trend YoY growth rate
    quart_gdp_data['Price Decomp','Adjusted',col,'Trend YoY Growth Rate'] = np.exp(series_trend).diff(4)/np.exp(series_trend).shift(4)

<a class="anchor" id="quart_gdp_contrib_qonq"></a>
## 2.6. Calculation of Additive Contributions to QonQ and YoY Growth 

In [542]:
#Prepare the required data (Price indices and Volume measures in monetary terms)

##Quarterly volume measures in monetary terms (unadjusted and seasonally and calendar adjusted)
quart_gdp_cli_money_adj = 0.01*quart_gdp_data['Accounts','Volume','Adjusted'].multiply(year_gdp_data.loc[2009,('Accounts','Nominal')]/4,axis=1)
quart_gdp_cli_money_noadj = 0.01*quart_gdp_data['Accounts','Volume','Not Adjusted'].multiply(year_gdp_data.loc[2009,('Accounts','Nominal')]/4,axis=1)
quart_gdp_cli_money = pd.concat([quart_gdp_cli_money_adj,quart_gdp_cli_money_noadj],axis=1,keys=['Adjusted','Not Adjusted'])

##Annual volume measures in monetary terms (derived by temporally aggregating quarterly measures)
###Unadjusted
year_gdp_cli_money_noadj = quart_gdp_cli_money_noadj.groupby('Year').sum()
####Add 1998 annual data for the identified aggregates (based on temporal consistency results)
for col in ['Final Private Consumption Expenditure','Final Domestic Consumption Expenditure','Final Domestic Demand Expenditure']:
    year_gdp_cli_money_noadj[col].loc[1998] = 0.01*year_gdp_data['Accounts','Volume',col].loc[1998]\
    *year_gdp_data['Accounts','Nominal',col].loc[2009]
####Mark 2024 annual data as missing
year_gdp_cli_money_noadj.loc[2024]=np.nan
###Calendar Adjusted
year_gdp_cli_money_adj = quart_gdp_cli_money_adj.groupby('Year').sum()
####Add 1998 annual data for the identified aggregates (based on temporal consistency results and using the temporally aggregated volume indices for temporally inconsistent series)
year_gdp_cli_money_adj['Final Private Consumption Expenditure'].loc[1998] = 0.01*year_gdp_data['Accounts','Volume Cal Adjusted','Final Private Consumption Expenditure'].loc[1998]\
    *year_gdp_data['Accounts','Nominal','Final Private Consumption Expenditure'].loc[2009]
for col in ['Final Domestic Consumption Expenditure','Final Domestic Demand Expenditure']:
    year_gdp_cli_money_adj[col].loc[1998] = 0.01*year_gdp_data['Accounts','Volume Cal Adjusted',f'{col} 2'].loc[1998]\
    *year_gdp_data['Accounts','Nominal',col].loc[2009]
####Mark 2024 annual data as missing
year_gdp_cli_money_adj.loc[2024]=np.nan
###Concatenate
year_gdp_cli_money = pd.concat([year_gdp_cli_money_noadj,year_gdp_cli_money_adj],axis=1,keys=['Not Adjusted','Adjusted'])

##Annual price indices (based on temporally aggregated quarterly measures) (unadjusted and calendar adjusted)
year_price_index_noadj = 100*(year_gdp_data['Accounts','Nominal']/year_gdp_cli_money_noadj)
year_price_index_adj = 100*(year_gdp_data['Accounts','Nominal']/year_gdp_cli_money_adj)
year_price_index = pd.concat([year_price_index_noadj,year_price_index_adj],axis=1,keys=['Not Adjusted','Adjusted'])

In [543]:
#Define a function that computes contributions to nominal and volume growth
def q_contrib(Comp,agg_name):
    
    global quart_gdp_data,quart_gdp_cli_money,year_gdp_cli_money,year_price_index
    
    
    Comp_0 = Comp
    #Contributions to QonQ and YoY growth
    for freq in ['QonQ','YoY']:
        Comp = Comp_0
        key_0 = f'Contribution to {agg_name} Growth {freq}'
        f = 1*(freq=='QonQ') + 4*(freq=='YoY')
        #Calculate contributions to nominal growth (unadjusted and seasonally adjusted)
        for adj in ['Not Adjusted','Seas Adjusted']:
            df = quart_gdp_data['Accounts','Nominal',adj][list(Comp)].diff(f).div(quart_gdp_data['Accounts','Nominal',adj,Comp[0]].shift(f),axis=0)
            #Adjust the sign for Imports' contributions
            if agg_name == 'GDP': df['Imports Of Goods And Services'] = - df['Imports Of Goods And Services']
            #Add to parent dataset
            df = pd.concat([df],axis=1,keys=[(key_0,'Nominal',adj)])
            quart_gdp_data = pd.concat([quart_gdp_data,df],axis=1)
        print(f'{(key_0,"Nominal")} has been added to the parent dataset')
        #Calculate contributions to volume growth (unadjusted and seasonally and calendar adjusted)
        if agg_name == 'GDP': Comp = tuple([i for i in Comp if i!='Change In stocks'])
        for adj in ['Not Adjusted','Adjusted']:
            df_0 = quart_gdp_cli_money[adj][list(Comp)].diff(f)*year_price_index[adj][list(Comp)].shift()\
            .div(quart_gdp_cli_money[adj,Comp[0]].shift(f)*year_price_index[adj,Comp[0]].shift(),axis=0)
            #Correction term
            df_co = (quart_gdp_cli_money[adj][list(Comp)].shift(f).div(quart_gdp_cli_money[adj,Comp[0]].shift(f),axis=0)-\
                      year_gdp_cli_money[adj][list(Comp)].shift().div(year_gdp_cli_money[adj,Comp[0]].shift(),axis=0))*\
                        (year_price_index[adj][list(Comp)].div(year_price_index[adj,Comp[0]],axis=0)).diff().shift()
            #Limit the correction term to the first quarter for QonQ growth
            if freq=='QonQ': df_co.loc[(slice(None),('Q2','Q3','Q4')),:] = 0
            df = df_0 + df_co
            #Calculate the contribution of Change in Stocks to GDP volume growth and adjust the sign for Imports' contributions
            if agg_name == 'GDP': 
                df['Change In stocks'] = df['GDP'] - (df['Final Domestic Demand Expenditure']+\
                                    df['Exports Of Goods And Services'] - df['Imports Of Goods And Services'])
                df['Imports Of Goods And Services'] = - df['Imports Of Goods And Services']
            #Add to parent dataset
            df = pd.concat([df],axis=1,keys=[(key_0,'Volume',adj)])
            quart_gdp_data = pd.concat([quart_gdp_data,df],axis=1)
        print(f'{(key_0,"Volume")} has been added to the parent dataset')

In [544]:
#Compute contributions to the growth of:
#GDP, Final Domestic Consumption Expenditure, and Final Domestic Demand Expenditure 
#(see A-III-6 in the Methods Section of the report)
S1 = tuple(quart_gdp_data['Accounts','Nominal','Not Adjusted'].columns)
S2 = ('Final Domestic Consumption Expenditure','Final Consumption Expenditure of Households',
        'Final Consumption Expenditure Of Npish','Final Private Consumption Expenditure',
    'Government Final Consumption Expenditure')
S3 = ('Final Domestic Demand Expenditure','Final Consumption Expenditure of Households',
        'Final Consumption Expenditure Of Npish','Final Private Consumption Expenditure',
    'Government Final Consumption Expenditure','Final Domestic Consumption Expenditure','Gross Fixed Capital Formation')

L = [(S1,'GDP'),(S2,'Domestic Consumption'),(S3,'Domestic Demand')]

for req in L:
    q_contrib(req[0],req[1])

('Contribution to GDP Growth QonQ', 'Nominal') has been added to the parent dataset
('Contribution to GDP Growth QonQ', 'Volume') has been added to the parent dataset
('Contribution to GDP Growth YoY', 'Nominal') has been added to the parent dataset
('Contribution to GDP Growth YoY', 'Volume') has been added to the parent dataset
('Contribution to Domestic Consumption Growth QonQ', 'Nominal') has been added to the parent dataset
('Contribution to Domestic Consumption Growth QonQ', 'Volume') has been added to the parent dataset
('Contribution to Domestic Consumption Growth YoY', 'Nominal') has been added to the parent dataset
('Contribution to Domestic Consumption Growth YoY', 'Volume') has been added to the parent dataset
('Contribution to Domestic Demand Growth QonQ', 'Nominal') has been added to the parent dataset
('Contribution to Domestic Demand Growth QonQ', 'Volume') has been added to the parent dataset
('Contribution to Domestic Demand Growth YoY', 'Nominal') has been added to t

 <a class="anchor" id="quart_gdp_share"></a>
## 2.7. Shares in Main Aggregates

In [545]:
#Prepare the required data (price indices and volume measures in monetary terms)

##Quarterly volume measures in monetary terms (unadjusted and seasonally and calendar adjusted)
quart_gdp_cli_money_adj = 0.01*quart_gdp_data['Accounts','Volume','Adjusted'].multiply(year_gdp_data.loc[2009,('Accounts','Nominal')]/4,axis=1)
quart_gdp_cli_money_noadj = 0.01*quart_gdp_data['Accounts','Volume','Not Adjusted'].multiply(year_gdp_data.loc[2009,('Accounts','Nominal')]/4,axis=1)
quart_gdp_cli_money = pd.concat([quart_gdp_cli_money_adj,quart_gdp_cli_money_noadj],axis=1,keys=['Adjusted','Not Adjusted'])
##Quarterly price indices
quart_price_index_adj = quart_gdp_data.loc[:,('Price Deflator','Adjusted',slice(None),'Index')].copy()
quart_price_index_adj.columns = quart_price_index_adj.columns.droplevel([0,3])
quart_price_index_noadj = quart_gdp_data.loc[:,('Price Deflator','Not Adjusted',slice(None),'Index')].copy()
quart_price_index_noadj.columns = quart_price_index_noadj.columns.droplevel([0,3])
quart_price_index = pd.concat([quart_price_index_adj,quart_price_index_noadj],axis=1)

In [546]:
#Define a function to calculate nominal shares as well as their volume and price components (see A-III-7 in the Methods Section of the report)
def q_share(Comp,agg_name):
    
    global quart_gdp_data,quart_price_index,quart_gdp_cli_money
    key_0 = f'Share in {agg_name}'
    #Calculate nominal shares
    for adj in ['Not Adjusted','Seas Adjusted']:
        df = quart_gdp_data['Accounts','Nominal',adj][list(Comp)].div(quart_gdp_data['Accounts','Nominal',adj,Comp[0]],axis=0)
        if agg_name == 'GDP': df['Imports Of Goods And Services'] = - df['Imports Of Goods And Services']
        df = pd.concat([df],axis=1,keys=[(key_0,'Nominal',adj)])
        quart_gdp_data = pd.concat([quart_gdp_data,df],axis=1)
        
    #Calculate shares' volume and price components
    if agg_name == 'GDP': Comp = tuple([i for i in Comp if i!='Change In stocks'])
    for db,key_1 in [(quart_gdp_cli_money,'Relative Volume'),(quart_price_index,'Relative Price')]:
        for adj in ['Adjusted','Not Adjusted']:
            df = db[adj][list(Comp)].div(db[adj,Comp[0]],axis=0)
            if agg_name == 'GDP' and key_1 == 'Relative Volume': 
                df['Imports Of Goods And Services'] = - df['Imports Of Goods And Services'] 
            df = pd.concat([df],axis=1,keys=[(key_0,key_1,adj)])
            quart_gdp_data = pd.concat([quart_gdp_data,df],axis=1)
    print('Done')

In [547]:
#Compute shares in:
#GDP, Final Domestic Consumption Expenditure, and Final Domestic Demand Expenditure 
#(see A-III-7 in the Methods Section of the report)
S1 = tuple(quart_gdp_data['Accounts','Nominal','Not Adjusted'].columns)
S2 = ('Final Domestic Consumption Expenditure','Final Consumption Expenditure of Households',
        'Final Consumption Expenditure Of Npish','Final Private Consumption Expenditure',
    'Government Final Consumption Expenditure')
S3 = ('Final Domestic Demand Expenditure','Final Consumption Expenditure of Households',
        'Final Consumption Expenditure Of Npish','Final Private Consumption Expenditure',
    'Government Final Consumption Expenditure','Final Domestic Consumption Expenditure','Gross Fixed Capital Formation')

L = [(S1,'GDP'),(S2,'Domestic Consumption'),(S3,'Domestic Demand')]

for req in L:
    q_share(req[0],req[1])

Done
Done
Done


 <a class="anchor" id="quart_gdp_stocks"></a>
## 2.8. Change in Stocks in Real Terms

In [548]:
#For unadjusted and adjusted series (nominal series do not contain calendar effects)
for adj in [('Not Adjusted','Not Adjusted'),('Seas Adjusted','Adjusted')]:
    #Calculate change in stocks in real terms
    series_r = 100*(quart_gdp_data['Accounts','Nominal',adj[0],'Change In stocks']/quart_gdp_data.loc[2009,('Accounts','Nominal',adj[0],'Change In stocks')].mean())\
/(0.01*quart_gdp_data['Price Deflator',adj[1],'GDP','Index'])
    #Calculate QonQ growth rate
    series_qonq = series_r.diff()/abs(series_r.shift())
    #Calculate YoY growth rate
    series_yoy = series_r.diff(4)/abs(series_r.shift(4))
    #Insert in parent dataset
    Series = [series_r,series_qonq,series_yoy]
    j = 0
    for data in ['Accounts','Growth Rate QonQ','Growth Rate YoY']:
        pos = quart_gdp_data.columns.get_loc((data,'Volume',adj[1],'Final Domestic Demand Expenditure'))+1
        quart_gdp_data.insert(pos,(data,'Volume',adj[1],'Change in Stocks (Real)'),Series[j])
        j += 1

### Save Data

file_name = curr_dir + '\\Results\\q_gdp_data.xlsx'
quart_gdp_data.to_excel(file_name)

file_name = curr_dir + '\\Results\\y_gdp_data.xlsx'
year_gdp_data.to_excel(file_name)